# Demo 6

In [ ]:
import pennylane as qml
from pennylane import qchem
import numpy as np
import matplotlib.pyplot as plt

## Use `qchem` to obtain Hamiltonian

In [ ]:
h, qubits = qchem.molecular_hamiltonian(
    name="1.10",
    geo_file="qchem/h2_1.10.xyz",  # molecular hydrogen
)

In [ ]:
type(h)

In [ ]:
h.terms

## Step two: define an ansatz circuit

In [ ]:
dev = qml.device("default.qubit", wires=qubits)

In [ ]:
electrons = 2

### Define the HF state

In [ ]:
ref_state = qchem.hf_state(electrons, qubits)

### Generate single and double excitations

In [ ]:
singles, doubles = qchem.excitations(electrons, qubits)

### Map excitations to wires

In [ ]:
s_wires, d_wires = qchem.excitations_to_wires(singles, doubles)

### Use chemically inspired UCCSD ansatz

In [ ]:
def circuit(params, wires):
    qml.templates.UCCSD(params, init_state=ref_state, s_wires=s_wires, d_wires=d_wires, wires=wires)
    # Note that we don't specify a return here!

### ALTERNATIVE: Use a hardware friendly ansatz

In [ ]:
def circuit(params, wires):
    
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

## Step three: set up cost function

In [ ]:
energy = qml.VQECost(circuit, h, dev)

In [ ]:
np.random.seed(0)
params = np.random.random(qubits)
params

In [ ]:
energy(params)

## Step four: minimize energy

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [ ]:
for i in range(100):
    params = opt.step(energy, params)
    
    if i % 10 == 0:
        print(energy(params))

In [ ]:
params

In [ ]:
dev.state

## Potential energy surface

In [ ]:
molec_structure_files = {  # keys: atomic separations (in Angstroms), values: corresponding file names
    0.3: "h2_0.30.xyz",
    0.5: "h2_0.50.xyz",
    0.7: "h2_0.70.xyz",
    0.9: "h2_0.90.xyz",
    1.1: "h2_1.10.xyz",
    1.3: "h2_1.30.xyz",
    1.5: "h2_1.50.xyz",
    1.7: "h2_1.70.xyz",
    1.9: "h2_1.90.xyz",
    2.1: "h2_2.10.xyz",
}

In [ ]:
hamiltonians = []

for separation, filename in molec_structure_files.items():

    h, nr_qubits = qchem.molecular_hamiltonian(
        name=str(separation),
        geo_file="qchem/" + filename,
    )
    hamiltonians.append(h)

In [ ]:
params_all = np.load("qchem/params_all.npy")

In [ ]:
params_all.shape

In [ ]:
ground_state_energies = []

for h, params in zip(hamiltonians, params_all):
    energy = qml.VQECost(circuit, h, dev)
    ground_state_energy = energy(params)
    ground_state_energies.append(ground_state_energy)

In [ ]:
plt.xlabel("Atomic separation (Angstrom)")
plt.ylabel("Energy (Ha)")
plt.plot(molec_structure_files.keys(), ground_state_energies)